In [1]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2


In [2]:
path = r'C:\Users\fajar\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK066096,Karma Resorts,Other,Past Guests,Karma Kandara,PF296726,Ms,Maricha,Mabunay,Female,...,2024-12-18 00:00:00,2,HOUSE_USE,V_OWNER_NRB,COMP,2,2,0,Email and Phone,NO
1,KB046143,Karma Resorts,Other,Past Guests,Karma Bavaria,PF296418,Mrs,Clement,Surgess,Female,...,2024-12-16 00:00:00,4,WYNDHAM,WYNDHAM,WYNDHAM,3,0,0,Email and Phone,NO
2,KK065230,Karma Resorts,Other,Past Guests,Karma Kandara,PF202839,Ms,Georgie,Smith,Female,...,2024-12-16 00:00:00,2,HOUSE_USE,V_OWNER_NRB,COMP,1,0,0,Email and Phone,NO
3,KK047983,Karma Resorts,Other,Past Guests,Karma Kandara,PF259088,Miss,Natalie,West,Female,...,2024-12-22 00:00:00,7,HOUSE_USE,CHAT,DIGITAL,6,0,0,Email and Phone,NO
4,KK063111,Karma Resorts,Other,Past Guests,Karma Kandara,PF291338,Mrs,Oriana,Mastro,Female,...,2024-12-18 00:00:00,2,HOUSE_USE,V_OWNER,COMP,6,0,0,Email Only,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,KK059814,Karma Resorts,Other,Past Guests,Karma Kandara,<NA>,Ms,Oxana,Khoroshavina,Unknown,...,2024-12-15 00:00:00,1,WEDDING,WEDDING,GROUP,2,0,0,Email Only,NO
61,KB039996,Karma Resorts,Other,Past Guests,Karma Bavaria,<NA>,Firma,Roche,Diagnostig,Unknown,...,2024-12-18 18:00:00,0,CB,CB,GROUP,1,0,0,Email Only,NO
62,KB039995,Karma Resorts,Other,Past Guests,Karma Bavaria,<NA>,Firma,Roche,Diagnostig,Unknown,...,2024-12-17 18:00:00,0,CB,CB,GROUP,1,0,0,Email Only,NO
63,KB046245,Karma Resorts,Other,Past Guests,Karma Bavaria,<NA>,Mr,Wmna,Tengateway,Unknown,...,2024-12-18 00:00:00,3,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email Only,NO


In [3]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [4]:
df_analytic.fillna('empty', inplace=True)

In [5]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [6]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

c:\Users\fajar\Documents\Python\Task Offline\past_guest_toni_app\../zoho_data_app\main_module.py:90: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


In [7]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
14,KB045800,Karma Resorts,Other,Past Guests,Karma Bavaria,PF263835,Herr,Marko,Babic,Male,...,2024-12-15 00:00:00,1,COMP,GER_OWE,COMP,1,0,0,Email and Phone,NO
24,KB044893,Karma Resorts,Other,Past Guests,Karma Bavaria,PF288697,empty,Michael,Schmid,Unknown,...,2024-12-15 00:00:00,1,OTA,OTA,RETAIL,5,0,0,Email and Phone,NO
26,KB039972,Karma Resorts,Other,Past Guests,Karma Bavaria,PF261853,Mr,Alan,Ridley,Male,...,2024-12-20 00:00:00,2,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email Only,NO
27,KB040427,Karma Resorts,Other,Past Guests,Karma Bavaria,PF080008,empty,Kea,Marketing Gmbh,Unknown,...,2024-12-15 18:30:00,0,LGR,LGR,GROUP,1,0,0,Email Only,NO
28,KB044893,Karma Resorts,Other,Past Guests,Karma Bavaria,PF288697,empty,Michael,Schmid,Unknown,...,2024-12-15 00:00:00,1,OTA,OTA,RETAIL,2,2,0,Email and Phone,NO
39,KK065672,Karma Resorts,Other,Past Guests,Karma Kandara,PF286588,Mrs,Olga,Zhuravskaya,Female,...,2024-12-15 00:00:00,5,DIGITAL,WEDDING,GROUP,6,0,0,Email and Phone,NO
47,KB046488,Karma Resorts,Other,Past Guests,Karma Bavaria,PF165494,Mr,Anton,Tardif,Male,...,2024-12-18 00:00:00,1,COMP,STAFF,COMP,2,0,0,Email and Phone,NO
56,KLM007420,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF294121,Mr,Adrian,Fatechand,Male,...,2024-12-17 00:00:00,2,COMP,STAFF,COMP,1,0,0,Email Only,NO
60,KK059814,Karma Resorts,Other,Past Guests,Karma Kandara,empty,Ms,Oxana,Khoroshavina,Unknown,...,2024-12-15 00:00:00,1,WEDDING,WEDDING,GROUP,2,0,0,Email Only,NO
61,KB039996,Karma Resorts,Other,Past Guests,Karma Bavaria,empty,Firma,Roche,Diagnostig,Unknown,...,2024-12-18 18:00:00,0,CB,CB,GROUP,1,0,0,Email Only,NO


In [8]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK066096,Karma Resorts,Other,Past Guests,Karma Kandara,PF296726,Ms,Maricha,Mabunay,Female,...,2024-12-18 00:00:00,2,HOUSE_USE,V_OWNER_NRB,COMP,2,2,0,Email and Phone,NO
1,KB046143,Karma Resorts,Other,Past Guests,Karma Bavaria,PF296418,Mrs,Clement,Surgess,Female,...,2024-12-16 00:00:00,4,WYNDHAM,WYNDHAM,WYNDHAM,3,0,0,Email and Phone,NO
2,KK065230,Karma Resorts,Other,Past Guests,Karma Kandara,PF202839,Ms,Georgie,Smith,Female,...,2024-12-16 00:00:00,2,HOUSE_USE,V_OWNER_NRB,COMP,1,0,0,Email and Phone,NO
3,KK047983,Karma Resorts,Other,Past Guests,Karma Kandara,PF259088,Miss,Natalie,West,Female,...,2024-12-22 00:00:00,7,HOUSE_USE,CHAT,DIGITAL,6,0,0,Email and Phone,NO
4,KK063111,Karma Resorts,Other,Past Guests,Karma Kandara,PF291338,Mrs,Oriana,Mastro,Female,...,2024-12-18 00:00:00,2,HOUSE_USE,V_OWNER,COMP,6,0,0,Email Only,NO
5,KK065959,Karma Resorts,Other,Past Guests,Karma Kandara,PF296432,Mr,Xinyan,Xin,Male,...,2024-12-16 00:00:00,2,OTA,HYPERGUEST,CHANNELS,9,0,0,Email and Phone,NO
6,KK065958,Karma Resorts,Other,Past Guests,Karma Kandara,PF296455,Mr,Xinyan,Xin,Male,...,2024-12-16 00:00:00,2,OTA,OTA,RETAIL,9,0,0,Email and Phone,NO
7,KK062865,Karma Resorts,Other,Past Guests,Karma Kandara,PF077477,Mr,Jesper,Solgaard,Male,...,2024-12-22 00:00:00,9,HOUSE_USE,V_OWNER,COMP,6,0,0,Email and Phone,NO
8,KB045525,Karma Resorts,Other,Past Guests,Karma Bavaria,PF292246,Frau,Alexandra,Koch,Female,...,2024-12-15 00:00:00,2,COMP,GER_OWE,COMP,2,1,0,Email and Phone,NO
9,KB045524,Karma Resorts,Other,Past Guests,Karma Bavaria,PF285336,Mrs,Deborah,Hillmann,Female,...,2024-12-15 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO


In [9]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [10]:
find_member_booking.to_csv(r'C:\Users\fajar\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [11]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
7,KK062865,Karma Resorts,Other,Past Guests,Karma Kandara,PF077477,Mr,Jesper,Solgaard,Male,...,2024-12-22 00:00:00,9,HOUSE_USE,V_OWNER,COMP,6,0,0,Email and Phone,NO
9,KB045524,Karma Resorts,Other,Past Guests,Karma Bavaria,PF285336,Mrs,Deborah,Hillmann,Female,...,2024-12-15 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
12,KB046422,Karma Resorts,Other,Past Guests,Karma Bavaria,PF252109,Frau,Heike,Speitmann,Female,...,2024-12-15 00:00:00,1,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
13,KB045801,Karma Resorts,Other,Past Guests,Karma Bavaria,PF263835,Herr,Marko,Babic,Male,...,2024-12-15 00:00:00,1,COMP,GER_OWE,COMP,2,0,2,Email and Phone,NO
14,KB045197,Karma Resorts,Other,Past Guests,Karma Bavaria,PF223912,empty,Christa,Schwenzer,Female,...,2024-12-21 00:00:00,1,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO


In [12]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [13]:
count_data_akhir = df_analytic.shape[0]

In [14]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK059814,Karma Resorts,Other,Past Guests,Karma Kandara,PF283757,Ms,Elena,Schukina,Female,...,2024-12-15 00:00:00,4,DIGITAL,WEDDING,GROUP,2,0,0,Email and Phone,NO
1,KK065168,Karma Resorts,Other,Past Guests,Karma Kandara,PF295021,Ms,Sonya,Farah Diba,Female,...,2024-12-15 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
2,KK065239,Karma Resorts,Other,Past Guests,Karma Kandara,PF283752,Ms,Maria,Voronova,Female,...,2024-12-15 00:00:00,1,WEDDING,WEDDING,GROUP,2,0,0,Email and Phone,NO
3,KK059814,Karma Resorts,Other,Past Guests,Karma Kandara,PF286588,Mrs,Olga,Zhuravskaya,Female,...,2024-12-15 00:00:00,5,WEDDING,WEDDING,GROUP,5,0,0,Email and Phone,NO
4,KK059814,Karma Resorts,Other,Past Guests,Karma Kandara,PF283760,Ms,Natalia,Tchemerisova,Female,...,2024-12-15 00:00:00,4,WEDDING,WEDDING,GROUP,2,1,0,Email and Phone,NO
5,KB044893,Karma Resorts,Other,Past Guests,Karma Bavaria,PF288697,empty,Michael,Schmid,Unknown,...,2024-12-15 00:00:00,1,OTA,OTA,RETAIL,2,1,0,Email and Phone,NO
6,KK059814,Karma Resorts,Other,Past Guests,Karma Kandara,PF295193,Mr,Alexander,Shumskiy,Male,...,2024-12-15 00:00:00,6,DIGITAL,WEDDING,GROUP,7,3,0,Email and Phone,NO
7,KB041862,Karma Resorts,Other,Past Guests,Karma Bavaria,PF178787,empty,Florian,Schofbeck,Male,...,2024-12-15 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
8,KB045820,Karma Resorts,Other,Past Guests,Karma Bavaria,PF293721,Frau,Pia,Melas,Female,...,2024-12-15 00:00:00,2,OTA,OTA,RETAIL,4,0,1,Email and Phone,NO
9,KK059814,Karma Resorts,Other,Past Guests,Karma Kandara,PF283750,Ms,Marina,Kushchina,Female,...,2024-12-15 00:00:00,6,WEDDING,WEDDING,GROUP,10,1,0,Email and Phone,NO


In [15]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [16]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


C:\Users\fajar\AppData\Local\Temp\ipykernel_20052\1513714307.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analytic['DOB'].replace('empty', '', inplace=True)


In [17]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK059814,Karma Resorts,Other,Past Guests,Karma Kandara,PF283757,Ms,Elena,Schukina,Female,...,2024-12-15,4,DIGITAL,WEDDING,GROUP,2,0,0,Email and Phone,NO
1,KK059814,Karma Resorts,Other,Past Guests,Karma Kandara,PF293355,Mr,Grigory,Pogosyan,Male,...,2024-12-15,4,COMP,WEDDING,GROUP,2,0,0,Email and Phone,NO
2,KK065618,Karma Resorts,Other,Past Guests,Karma Kandara,PF283758,Ms,Oxana,Khoroshavina,Female,...,2024-12-15,1,WEDDING,WEDDING,GROUP,2,1,0,Email and Phone,NO
3,KSH021393,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF294505,,Richard,Barnes,Unknown,...,2024-12-15,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
4,KSH021532,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF295133,,Louise,Lloyd-Smith,Unknown,...,2024-12-15,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
5,KSH021678,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF296078,,Zac,White,Unknown,...,2024-12-15,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
6,KLM005562,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF273457,Ms,Jacqueline,Eyre,Unknown,...,2024-12-15,2,DIR,OTA,RETAIL,2,0,0,Email and Phone,NO
7,KLM007509,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF295179,Mr,Neil,Morton,Unknown,...,2024-12-15,2,DIGITAL,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
8,KB045525,Karma Resorts,Other,Past Guests,Karma Bavaria,PF292246,Frau,Alexandra,Koch,Female,...,2024-12-15,2,COMP,GER_OWE,COMP,2,1,0,Email and Phone,NO
9,KB046244,Karma Resorts,Other,Past Guests,Karma Bavaria,PF267999,Mrs,Angela Karnes,Padron,Female,...,2024-12-15,2,WYNDHAM,WYNDHAM,WYNDHAM,2,0,1,Email and Phone,NO


In [18]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 65
Jumlah Data Yang Duplicate : 12
Jumlah Data Setelah Hapus Duplicate : 53
Jumlah Data Yang Termasuk Member: 5
Jumlah Data Setelah Hapus Member : 48
Jumlah Data Akhir : 48


In [19]:
df_analytic.to_excel(r'C:\Users\fajar\Documents\Python\Data\final_analytics.xlsx', index=False)